In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed=42

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *

import gc

np.random.seed(seed)

PATH = Path('data_draw')

# reset learner
def reset_learner(data,is_fp16=False):
    learn=None
    gc.collect()
    learn = create_cnn(data, models.resnet50, metrics=accuracy)
    if is_fp16:
        learn = learn.to_fp16()
    return learn
def get_data(bs,size,tfms,path=PATH,seed=seed):
    np.random.seed(seed)
#     tfms = get_transforms(do_flip=True, max_rotate=5, max_zoom=1, max_lighting=0.2, max_warp=0,p_affine=0,p_lighting=0.75)
    return ImageDataBunch.from_folder(path,train='train',valid='valid',bs=bs,ds_tfms = tfms,size=size,num_workers=4).normalize(imagenet_stats)



# stage 1 size 300

In [3]:
# default
data=None
gc.collect()

tfms = get_transforms()
data = get_data(78,300,tfms)
learn= reset_learner(data,is_fp16=True)

In [4]:
learn.freeze()
learn.fit_one_cycle(8,max_lr=8e-03)

epoch,train_loss,valid_loss,accuracy
1,0.443387,0.429400,0.864198
2,0.438887,0.553943,0.827932
3,0.363468,0.691960,0.802469
4,0.279841,0.275809,0.905093
5,0.196060,0.165789,0.951389
6,0.126328,0.070487,0.976852
7,0.095652,0.049324,0.985340
8,0.075177,0.045223,0.986111


In [6]:
# learn.save('final-stage1-300-new-8epochs');

# stage 2 size 300

In [7]:
tfms = get_transforms()
data=None
gc.collect()
data = get_data(64,300,tfms)
learn= reset_learner(data,is_fp16=True)

In [ ]:
learn.load('final-stage1-300-new-8epochs');

In [8]:
old_lr = 8e-03
learn.unfreeze()
learn.fit_one_cycle(5,max_lr=slice(8e-06,old_lr/5))

epoch,train_loss,valid_loss,accuracy
1,0.291655,0.236876,0.919753
2,0.156573,0.116325,0.966049
3,0.088474,0.032272,0.990741
4,0.030241,0.010092,0.995370
5,0.016753,0.003159,0.999228


In [10]:
# learn.save('final-stage2-300-new-5epochs');

# stage 1 size 350

In [3]:
data=None
gc.collect()

tfms = get_transforms()
data = get_data(60,350,tfms)
learn= reset_learner(data,is_fp16=True)
learn.load('final-stage2-300-new-5epochs');

In [5]:
# learn.freeze()
# learn.fit_one_cycle(5,max_lr=3e-03)
# learn.save('final-stage1-350-new-5epochs-303');

epoch,train_loss,valid_loss,accuracy
1,0.041577,0.020128,0.993827
2,0.064348,0.015572,0.993827
3,0.048684,0.019544,0.996142
4,0.032033,0.004563,0.997685
5,0.029207,0.004492,0.998457


In [4]:
learn.freeze()
learn.fit_one_cycle(8,max_lr=3e-03)
# learn.save('final-stage1-350-new-8epochs-303');

epoch,train_loss,valid_loss,accuracy
1,0.036529,0.021894,0.995370
2,0.093634,0.092346,0.969136
3,0.088372,0.040983,0.988426
4,0.073058,0.081078,0.973765
5,0.048307,0.024442,0.993827
6,0.032650,0.009546,0.996914
7,0.018685,0.007257,0.997685
8,0.018920,0.007084,0.998457
